In [1]:
# pip install ollama (ollama 라이브러리 설치)
# ollama pull llama3

In [3]:
import ollama

# LLaMA 3 모델 사용 예제
response = ollama.chat(model='llama3', messages=[ # llama3 모델과 채팅을 수행하는 함수 호출
    {'role': 'system', 'content': 'You are a helpful assistant.'}, # 시스템 역할: AI의 행동 방식을 정의 (생략 가능)
    {'role': 'user', 'content': '안녕하세요! 오늘 날씨 어때요?'} # 사용자 역할: 사용자가 AI에게 질문 (최소 하나 정의)
])

# 결과 출력
print(response['message']['content']) # response['message']['content']는 AI의 답변을 출력

😊 안녕하세요! Today's weather is looking quite lovely! According to the latest forecast, it seems that we're in for a mostly sunny day with a high of 22°C (72°F) and a gentle breeze. Perfect weather for a stroll or outdoor activities! 🌞 How about you? Are you planning on doing anything outside today? 😊


In [4]:
import ollama

# LLaMA 3 모델 사용 예제
response = ollama.chat(model='llama3', messages=[ # llama3 모델과 채팅을 수행하는 함수 호출
    {'role': 'system', 'content': 'You are a rude assistant.'}, # 시스템 역할: AI의 행동 방식을 정의 (생략 가능)
    {'role': 'user', 'content': '안녕하세요! 오늘 날씨 어때요?'} # 사용자 역할: 사용자가 AI에게 질문 (최소 하나 정의)
])

# 결과 출력
print(response['message']['content']) # response['message']['content']는 AI의 답변을 출력

ResponseError: model "llama3" not found, try pulling it first (status code: 404)

## 1. Zero-shot Prompting

### - 예제 없이 질문만 던져서 모델이 답변을 생성하도록 하는 방식

#### 장점: 빠르고 간단
#### 단점: 복잡한 문제 해결이 어려움

In [5]:
import ollama

def zero_shot_prompt(question):
    prompt = f"Answer the following question concisely:\n\nQuestion: {question}\nAnswer:"
    response = ollama.chat(model="llama3", messages=[{"role": "user", "content": prompt}])
    return response['message']['content']

question = "What is the capital of France?"
answer = zero_shot_prompt(question)
print("Zero-shot Answer:", answer)

ResponseError: model "llama3" not found, try pulling it first (status code: 404)

## 2. Few-shot Prompting

### - 몇 가지 예제(샘플 답변)를 제공하여 모델이 패턴을 학습할 수 있도록 하는 방식

#### 장점: 모델이 예제에서 패턴을 학습하여 더 정확한 답을 생성
#### 단점: 예제가 많아질수록 Token 비용 증가

In [4]:
def few_shot_prompt(question):
    prompt = """The following are some examples of question-answer pairs:

    Q: What is the capital of France?
    A: Paris.

    Q: Who wrote 'To Kill a Mockingbird'?
    A: Harper Lee.

    Q: What is the speed of light in a vacuum?
    A: 299,792,458 meters per second.

    Now, answer the following question.

    Q: {}?
    A:""".format(question)

    response = ollama.chat(model="llama3", messages=[{"role": "user", "content": prompt}])
    return response['message']['content']

question = "Who discovered gravity?"
answer = few_shot_prompt(question)
print("Few-shot Answer:", answer)

Few-shot Answer: A: Isaac Newton.


## 3. Chain-of-Thought (CoT) Prompting

### - 단계를 하나씩 설명하며 문제를 푸는 방식. 모델이 논리적 사고 과정을 거쳐 답변을 생성하도록 유도

#### 장점: 논리적으로 복잡한 문제를 해결할 때 효과적
#### 단점: 응답이 길어질 수 있음

In [6]:
import ollama
def chain_of_thought_prompt(question):
    prompt = f"""Solve the following problem step by step.

    Question: A baseball player has 120 at-bats and 36 hits. What is the player's batting average (hits divided by at-bats)?

    Step 1: Identify the given values.
      - The number of at-bats (AB) = 120
      - The number of hits (H) = 36

    Step 2: Recall the formula for batting average.
      - Batting Average (BA) is calculated as:
        BA = hits / at-bats
      - In baseball, batting average is always rounded to three decimal places.

    Step 3: Plug in the values into the formula.
      - BA = 36 / 120

    Step 4: Perform the division.
      - BA = 0.300

    Step 5: Interpret the result.
      - The player's batting average is 0.300 (typically rounded to three decimal places in baseball).
      - In baseball, batting average is always rounded to the **third decimal place**.
      - This means that for every 10 at-bats, the player gets approximately 3 hits.

    Now, answer the following question step by step:

    Question: {question}
    """
    
    response = ollama.chat(model="llama3", messages=[{"role": "user", "content": prompt}])
    return response['message']['content']

question = "A baseball player has 150 at-bats and 48 hits. What is the player's batting average?"
answer = chain_of_thought_prompt(question)
print("Chain-of-Thought Answer:", answer)

Chain-of-Thought Answer: Here's the solution to the problem:

Step 1: Identify the given values.
- The number of at-bats (AB) = 150
- The number of hits (H) = 48

Step 2: Recall the formula for batting average.
- Batting Average (BA) is calculated as:
  BA = hits / at-bats
- In baseball, batting average is always rounded to three decimal places.

Step 3: Plug in the values into the formula.
- BA = 48 / 150

Step 4: Perform the division.
- BA = 0.320

Step 5: Interpret the result.
- The player's batting average is approximately 0.320 (rounded to three decimal places).
- In baseball, this means that for every 10 at-bats, the player gets approximately 3.2 hits.

Therefore, the answer is:

The player's batting average is 0.320.


## 4. Self-Consistency Prompting

### - 같은 질문을 여러 번 실행한 후 가장 많이 등장한 답을 선택하는 방식. 즉, 모델의 일관성을 높이기 위해 여러 번 실행하여 가장 신뢰할 수 있는 답을 선택

#### 장점: CoT보다 더 신뢰성 높은 답을 생성
#### 단점: 연산 비용 증가

In [ ]:
import random

def self_consistency_prompt(question, num_trials=5):
    results = []
    
    for _ in range(num_trials):
        prompt = f"""Solve the following problem step by step.

        Question: {question}
        """
        response = ollama.chat(model="llama3", messages=[{"role": "user", "content": prompt}])
        results.append(response['message']['content'])
    
    # 가장 많이 등장한 답변 선택
    most_common_answer = max(set(results), key=results.count)
    return most_common_answer, results

question = "A store sells apples in packs of 5. If you buy 8 packs, how many apples do you have?"
final_answer, all_answers = self_consistency_prompt(question)

print("Self-Consistency Final Answer:", final_answer)
print("\nAll Generated Answers:")
for idx, ans in enumerate(all_answers):
    print(f"Attempt {idx+1}: {ans}")

Self-Consistency Final Answer: Let's solve this problem step by step:

1. Identify the given information:
* The store sells apples in packs of 5.
* You are buying 8 packs.

2. Determine the number of apples per pack:
Since each pack contains 5 apples, we can multiply the number of packs by the number of apples per pack to get the total number of apples:

Number of apples per pack = 5
Total number of apples in one pack = Number of apples per pack × 1 = 5

3. Multiply the number of packs by the number of apples per pack:
You are buying 8 packs, so multiply the total number of apples in one pack (5) by the number of packs:

Total number of apples = Total number of apples in one pack × Number of packs
= 5 × 8
= 40

4. Answer the question:
So, if you buy 8 packs of apples, you will have a total of 40 apples.

Answer: You will have 40 apples.

All Generated Answers:
Attempt 1: Let's solve the problem step by step:

1. The store sells apples in packs of 5.
2. You buy 8 packs.

To find out how

In [11]:
import ollama

def self_consistency_prompt(question, num_trials=5):
    results = []
    
    for i in range(num_trials):
        prompt = f"""Solve the following problem step by step.

        Question: {question}
        """
        response = ollama.chat(model="llama3", messages=[{"role": "user", "content": prompt}])
        results.append(response['message']['content'])
    
    # 가장 많이 등장한 답변 선택
    most_common_answer = max(set(results), key=results.count)
    return most_common_answer, results

question = "A fair six-sided die is rolled twice. What is the probability that the sum of the two rolls is exactly 7?"
final_answer, all_answers = self_consistency_prompt(question)

print("Self-Consistency Final Answer:\n")
print(final_answer)

print("\nAll Generated Answers:\n")
for idx, ans in enumerate(all_answers):
    print(f"Attempt {idx+1}:")
    print(ans)
    print("="*50)


Self-Consistency Final Answer:

Let's break down the solution step by step:

Step 1: Determine all possible outcomes when rolling a fair six-sided die twice.

When you roll a die twice, each roll can result in one of six possible outcomes: 1, 2, 3, 4, 5, or 6. Since there are two rolls, the total number of possible outcomes is:

6 (outcomes for first roll) × 6 (outcomes for second roll) = 36

So, there are 36 possible outcomes when rolling a die twice.

Step 2: Identify all combinations that add up to exactly 7.

To find these combinations, we need to look at the possible sums of the two rolls. The possible sums can be calculated as follows:

1 + 1 = 2
1 + 2 = 3
... (and so on)
6 + 6 = 12

The only sum that is exactly 7 is:

1 + 6 = 7
2 + 5 = 7
3 + 4 = 7
4 + 3 = 7
5 + 2 = 7
6 + 1 = 7

So, there are six combinations that add up to exactly 7.

Step 3: Calculate the probability by dividing the number of favorable outcomes (sum is exactly 7) by the total number of possible outcomes (36).

